In [44]:
import pandas as pd
from ics import Calendar, Event, DisplayAlarm
from datetime import datetime, timedelta

# read scraped output
df = pd.read_csv("output.csv", names=["url","artist_country","artist","location","date_string","insta","spotify"])

# read ratings and comments from google sheet
SHEET_ID = '1uu3_S_e2zR5O2cTbknIUh4bG1GymBtdwieFwZynTPRY'
SHEET_NAME = 'transform'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
df_ratings = pd.read_csv(url,usecols=['🧑‍🎤 artist 👩‍🎤','⭐ Mikael rating','⭐ Line rating','💭 comment'])

# rename
df_ratings = df_ratings.rename(columns={'🧑‍🎤 artist 👩‍🎤':'artist','⭐ Mikael rating':'m_rating','⭐ Line rating':'l_rating','💭 comment':'comment'})

In [45]:
# join website info with ratings 
df = df.join(df_ratings, how='outer', lsuffix='_left', rsuffix='_right')


In [46]:
# Extract day, month, and year into separate columns
#df[['day', 'month', 'year']] = df['date_string'].str.extract(r'(\d{1,2})\.\s*(\w+)\s*(\d{4})')
df['start_time'] = df['date_string'].str.extract(r'(\d{2}:\d{2})')

df['new_date_string'] = df['date_string'].replace(to_replace=r'.*dag (\d{1,2})\. (\w{3})\w', value=r'\1 \2', regex=True)
date_format = '%d %b %Y'

# Convert the date string to datetime format for ical
df['datetime'] = pd.to_datetime(df['new_date_string'], format=date_format)

## todo handle time of day for ical. Example below
# df[['date_string','new_date_string','datetime']][135:].head()

In [48]:
from datetime import datetime, timedelta
# write most important concerts to ical
c = Calendar()
for index, row in df[df['m_rating'] == '🤯JA!'].iterrows():
    e = Event()
    e.name = row['artist_left']
    e.begin = row['datetime']
    e.description = 'description_test'
    e.location = 'location_test'
    a=DisplayAlarm()
    a.trigger=timedelta(seconds=1)
    a.display_text=" tide alert!"
    e.alarms=[a]
    c.events.add(e)

# c.events
with open('most_important.ics', 'w') as my_file:
    my_file.writelines(c.serialize_iter())


/Users/mikaelthorup/repos/RF23/venv/lib/python3.10/site-packages/ics/component.py:85: FutureWarning: Behaviour of str(Component) will change in version 0.9 to only return a short description, NOT the ics representation. Use the explicit Component.serialize() to get the ics representation.
  warnings.warn(


import pandas as pd
from ics import Calendar, Event, DisplayAlarm

# Create a DataFrame with some events
df = pd.DataFrame({
    'name': ['Event 1', 'Event 2', 'Event 3'],
    'start': ['2022-05-01 10:00:00', '2022-05-02 14:30:00', '2022-05-03 18:00:00'],
    'end': ['2022-05-01 11:30:00', '2022-05-02 16:00:00', '2022-05-03 20:00:00'],
    'description': ['Description 1', 'Description 2', 'Description 3'],
    'location': ['Location 1', 'Location 2', 'Location 3']
})

# Create a Calendar object
c = Calendar()

# Iterate over the rows of the DataFrame and add each event to the Calendar
for index, row in df.iterrows():
    e = Event()
    e.name = row['name']
    e.begin = row['start']
    e.end = row['end']
    e.description = row['description']
    e.location = row['location']
    a = DisplayAlarm()
    a.trigger = '-PT10M'  # 10 minutes before the event
    a.action = 'DISPLAY'
    e.alarms.add(a)
    c.events.add(e)

# Write the Calendar to an ICS file
with open('my.ics', 'w') as my_file:
    my_file.writelines(c.serialize())

print('Events written to my.ics')


from ics import Calendar

with open('test.ics', 'r') as my_file:
    calendar_str = my_file.read()
calendar = Calendar(calendar_str)

for event in calendar.events:
    print(event.name)
    print(event.begin)
    print(event.end)
    print(event.description)
    print(event.location)
    for alarm in event.alarms:
        print(alarm.trigger)
